In [3]:
import numpy as np
import itertools
import math
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
import heapq
import pandas as pd

# Problem statement

<p float="left">
  <img src="https://i.imgur.com/nDe6qQB.png" width="500" />
  <img src="https://i.imgur.com/o7ruAe7.png" width="600" />
</p>

<img src="https://i.imgur.com/3R1W4Ln.png" width="800" />

Here's a breakdown of how your profit from an expedition will be computed:
Every spot has its **treasure multiplier** (up to 100) and the number of **inhabitants** (up to 8). The spot's total treasure is the product of the **base treasure** (10000, same for all spots) and the spot's specific treasure multiplier. However, the resulting amount is then divided by the sum of the hunters and the percentage of all the expeditions (from other players) that took place there. For example, if a field has 5 hunters, and 10% of all the expeditions (from all the other players) are also going there, the prize you get from that field will be divided by 15. After the division, **expedition costs** apply (if there are any), and profit is what remains.

Second expedition is optional: you are not required to do both. Fee for embarking upon a second expedition is 50 000. Order of submitted expeditions does not matter for grading.

# Solution

The profit for an expedition to a destination with $M$ as multiplier, $H$ inhabitants, and share $p\in [0,1]$ of all expeditions is:
$$\begin{cases} 
10000\frac M{H+100 p} &\text{if this is the first expedition,}  
\\ 10000\frac M{H+100 p} - 50000 &\text{if this is the second expedition,} 
\end{cases} 
$$
The second expedition is profitable if and only if $p < \frac{M - 5 H}{500}$.  
Let us see from the data how stringent these conditions are on $p$.

In [18]:
# collecting all the cases 
arr = np.array([(10,1), (80,6), (37,3), (17,1), (31,2), (90,10), (50,4), (20,2), (73,4), (89,8)])

The next cell computes the array with entries $\frac{3M - 10 H}{1000}$.

In [ ]:
thresh = np.zeros((10))
for i in range(10):
        mult, hunt = arr[i]
        thresh[i] = (mult - 5*hunt)/500
with np.printoptions(precision=3, suppress=True):
    # if the probability for visiting the ith block is thresh[i], then i is profitable
    print(thresh)



[0.01  0.1   0.044 0.024 0.042 0.08  0.06  0.02  0.106 0.098]


## 1. Maximin approach

The shares are computed once the choices of destinations are collected over all the teams.  
However, since the number of teams is large, our choice has very little impact on the shares.

One way to tackle the uncertainty in the share $p$ is to consider the most pessimistic setting where $p$ is chosen by a malicious adversary in order to make our profit as small as possible.  
For a choice of a single destination $(M,H)$, our profit $\pi(M,H,p)$ rewrites $\pi(M,H,p^*(M,H))$ where $p^*(M,H)$ is a solution of 
$$\min_{p\in [0,1]}  \frac {10000M}{H+100p}.$$
For a single expedition, the complete optimization problem is therefore 
$$\max_{(M,H)} \min_{p\in [0,1]}  \frac {10000M}{H+100 p} = \max_{(M,H)}  \frac {10000M}{H+100}$$

In [46]:
def maximin1():
    """Solves the maximin optimization problem for a single expedition.

    Parameters
    ----------
    
    Returns
    -------
    argmax : list of tuple
        Maximizers.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    argmax = []
    for mult, hunt in arr:
        val = (mult / (hunt + 100)) * 10000
        if math.isclose(val, max_val):
            argmax.append((mult, hunt))
        elif val > max_val:
            argmax = [(mult, hunt)]
            max_val = val
    return (argmax, max_val)

In [47]:
maximin1()

([(89, 8)], 8240.74074074074)

For a total of two expeditions, the optimization problem is 
$$\max_{(M_1,H_1), (M_2,H_2)} \quad \min_{\substack{p_1, p_2\in [0,1]\\ p_1+p_2\leq 1}}  \frac {10000M_1}{H_1+100 p_1}+\frac {10000M_2}{H_2+100 p_2}-50000.$$
Since the inner minimization problem is no longer trivial, we make a call to the Mathematica function `NMinimize` to solve it.

In [54]:

def maximize2():
    """Solves the maximin optimization problem for two expeditions. Returns only one solution, there may be other.

    Parameters
    ----------
    
    Returns
    -------
    argmax1 : tuple
        First expedition.
    argmax2 : tuple
        Second expedition.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    max_mult1, max_hunt1 = None, None
    max_mult2, max_hunt2 = None, None
    for (mult1, hunt1), (mult2, hunt2) in itertools.combinations(arr, 2):
        val_temp = float('inf')
        for p1 in range(0,100):
            for p2 in range(0,100):
                if (p1 + p2) > 100: break
                val_temp = min(val_temp, 
                               ((mult1/(hunt1+p1)) + (mult2/(hunt2+ p2))))
        # val_temp = session.evaluate(wlexpr(f'NMinimize[{{{mult1}/({hunt1} + 100*p1) + {mult2}/({hunt2} + 100*p2), 0 <= p1, 0 <= p2, p1 + p2 <= 1}}, {{p1, p2}}]'))[0]
        val = -50000 + val_temp * 10000
        print((mult1,hunt1), (mult2,hunt2), val_temp, val)
        if math.isclose(val, max_val):
            print("collision")
        if val > max_val:
            max_mult1, max_hunt1 = mult1, hunt1
            max_mult2, max_hunt2 = mult2, hunt2
            max_val = val
    return ((max_mult1, max_hunt1), (max_mult2, max_hunt2), max_val)

In [55]:
maximize2()

(10, 1) (80, 6) 1.3698010849909585 -36301.98915009041
(10, 1) (37, 3) 0.8218954248366013 -41781.04575163399
(10, 1) (17, 1) 0.5203761755485893 -44796.238244514105
(10, 1) (31, 2) 0.73996723996724 -42600.3276003276
(10, 1) (90, 10) 1.4414802065404475 -35585.197934595526
(10, 1) (50, 4) 0.997431506849315 -40025.684931506854
(10, 1) (20, 2) 0.5658914728682171 -44341.08527131783
(10, 1) (73, 4) 1.3051948051948052 -36948.05194805194
(10, 1) (89, 8) 1.455736224028907 -35442.63775971093
(80, 6) (37, 3) 2.0716783216783217 -29283.216783216783
(80, 6) (17, 1) 1.595890410958904 -34041.09589041096
(80, 6) (31, 2) 1.9501274117218783 -30498.72588278122
(80, 6) (90, 10) 2.928571428571429 -20714.28571428571
(80, 6) (50, 4) 2.3318835731013716 -26681.164268986286
(80, 6) (20, 2) 1.6666666666666667 -33333.33333333333
(80, 6) (73, 4) 2.7804232804232805 -22195.767195767196
(80, 6) (89, 8) 2.963020030816641 -20369.79969183359
(37, 3) (17, 1) 1.001536098310292 -39984.63901689708
(37, 3) (31, 2) 1.29272727272

((90, 10), (89, 8), -19661.016949152545)

When proceeding with two expeditions, you only lose money.  
From the pessimistic maximin standpoint, it is better to go for a single expedition.

## 2. Optimization given a prior

A less pessimistic approach is to model the shares *a priori*.
For each destination, the corresponding share is no longer unknown. 
Instead we posit a value for this share.  
Shares are nonnegative and sum to $1$, thus shares are modelled by a probability distribution that we call a prior.

Given a prior, the optimization problem is 
$$\max_{L\in \{1,2,3\}} \quad \max_{(M_i,H_i,p_i)} \text{fee}(L) + \sum_{i=1}^L \frac{10000 M_i}{H_i + 100p_i}   $$

### a. Natural prior

The ratio $M/H$ seems like a plausible proxy for the attractiveness of a destination.  
We compute the array `ratios` with entries $\frac MH$ and normalize it to obtain the prior array `shares` on the distribution of expeditions.

In [57]:
ratios = np.zeros((10))
for i in range(10):
        mult, hunt = arr[i]
        ratios[i] = mult/hunt
with np.printoptions(precision=2, suppress=True):
    print("Ratios:", ratios, sep='\n')

Ratios:
[10.   13.33 12.33 17.   15.5   9.   12.5  10.   18.25 11.12]


In [58]:
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior:", shares, sep='\n')

Natural prior:
[0.077 0.103 0.096 0.132 0.12  0.07  0.097 0.077 0.141 0.086]


`shares[0, 0] = 0.077` means that we posit that $7.7\%$ of all expeditions will go to the destination with multiplier $10$ and $1$ inhabitant.

The difference between larger and smaller values in `ratios` can be accentuated by elevating all the entries of `ratios` to some power.

In [59]:
ratios = np.zeros(10)
for i in range(10):
        mult, hunt = arr[i]
        ratios[i] = mult/hunt
ratios = ratios**5
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior with exponent 5:", shares, sep='\n')

Natural prior with exponent 5:
[0.017 0.073 0.049 0.246 0.155 0.01  0.053 0.017 0.35  0.029]


The next cell solves the optimization problem by iterating over the whole state space.

In [ ]:
def fee(n):
    """Compute the fee for a total of n expeditions.

    Parameters
    ----------
    n : int
        Number of expeditions.
    
    Returns
    -------
    float
        Fee.
    """
    if n == 1:
        return 0
    if n == 2:
        return -50000
    if n == 3:
        return -100000000 # discourage 3 since not allowed
def payoff(mults, hunts, shares):
    """Compute the final profit after the expeditions.

    Parameters
    ----------
    mults : list of int
        Multipliers for each destination.
    hunts : list of int
        Hunters for each destination.
    shares : list of int
        Shares for each destination.
    
    Returns
    -------
    float
        Profit.
    """
    return 10000 * sum([mult/(hunt + 100*share) for (mult, hunt, share) in zip(mults, hunts, shares)]) + fee(len(mults))

def maximize_prior_top(shares, k):
    """Given the prior, compute solutions that yield top k profits.

    Parameters
    ----------
    shares : list of int
        Shares for each destination.
    k : int
        Number of solutions
    
    Returns
    -------
    list of tuple
        Top k profits and optimal expeditions.
    """
    datas = [(mult, hunt, share) for ((mult, hunt), share) in zip(arr, shares)]
    heap = []
    iterables = [itertools.combinations(datas, n_exp) for n_exp in range(1, 4)]
    for (i, data) in enumerate(itertools.chain.from_iterable(iterables)):
        mults = [tupl[0] for tupl in data]
        hunts = [tupl[1] for tupl in data]
        shares = [tupl[-1] for tupl in data]
        val = payoff(mults, hunts, shares)
        expeditions = list(zip(mults, hunts))
        if i < k:
            heapq.heappush(heap, (val, expeditions))
        elif val > heap[0][0]:
            heapq.heappop(heap)
            heapq.heappush(heap, (val, expeditions))
    return sorted(heap, reverse=True)

Next, we consider the priors obtained by elevating the entries of `ratios` to an exponent `i`.   
When `i=0`, the prior is uniformly distributed over all destinations.

In [ ]:
for i in range(10):
    shares = ratios**i
    shares = shares / np.sum(shares)
    res = maximize_prior_top(shares, 20)
    print("Exponent:", i, "Profit:", f"{res[0][0]:.2f}", "Optimal expeditions:", res[0][1])

Exponent: 0 Profit: 52142.86 Optimal expeditions: [(80, 6), (73, 4)]
Exponent: 1 Profit: 112950.96 Optimal expeditions: [(90, 10), (89, 8)]
Exponent: 2 Profit: 150937.16 Optimal expeditions: [(80, 6), (89, 8)]
Exponent: 3 Profit: 188807.03 Optimal expeditions: [(80, 6), (50, 4)]
Exponent: 4 Profit: 203913.93 Optimal expeditions: [(80, 6), (50, 4)]
Exponent: 5 Profit: 207401.27 Optimal expeditions: [(80, 6), (50, 4)]
Exponent: 6 Profit: 208140.55 Optimal expeditions: [(80, 6), (50, 4)]
Exponent: 7 Profit: 217912.87 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 8 Profit: 228373.52 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 9 Profit: 233692.40 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 10 Profit: 236222.72 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 11 Profit: 237385.53 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 12 Profit: 237910.54 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 13 Profit: 238145.42 Optimal expeditions: [(80, 6), (31, 2)]
Exponent: 14 Pro

C:\Users\gdas2\AppData\Local\Temp\ipykernel_11600\1002411719.py:2: RuntimeWarning: overflow encountered in power
  shares = ratios**i
C:\Users\gdas2\AppData\Local\Temp\ipykernel_11600\1002411719.py:3: RuntimeWarning: invalid value encountered in divide
  shares = shares / np.sum(shares)


In [79]:
# using the multiplier as a priori
ratios = np.zeros((10))
for i in range(10):
        mult,_= arr[i]
        ratios[i] = mult
with np.printoptions(precision=2, suppress=True):
    print("Ratios:", ratios, sep='\n')

shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior:", shares, sep='\n')

for i in range(10):
    shares = ratios**i
    shares = shares / np.sum(shares)
    res = maximize_prior_top(shares, 20)
    print("Exponent:", i, "Profit:", f"{res[0][0]:.2f}", "Optimal expeditions:", res[0][1])

Ratios:
[10. 80. 37. 17. 31. 90. 50. 20. 73. 89.]
Natural prior:
[0.02  0.161 0.074 0.034 0.062 0.181 0.101 0.04  0.147 0.179]
Exponent: 0 Profit: 52142.86 Optimal expeditions: [(80, 6), (73, 4)]
Exponent: 1 Profit: 39062.23 Optimal expeditions: [(73, 4)]
Exponent: 2 Profit: 118042.44 Optimal expeditions: [(10, 1), (17, 1)]
Exponent: 3 Profit: 190483.33 Optimal expeditions: [(17, 1), (31, 2)]
Exponent: 4 Profit: 240138.47 Optimal expeditions: [(17, 1), (31, 2)]
Exponent: 5 Profit: 261738.22 Optimal expeditions: [(17, 1), (31, 2)]
Exponent: 6 Profit: 270150.76 Optimal expeditions: [(17, 1), (31, 2)]
Exponent: 7 Profit: 273256.80 Optimal expeditions: [(17, 1), (31, 2)]
Exponent: 8 Profit: 274377.61 Optimal expeditions: [(17, 1), (31, 2)]
Exponent: 9 Profit: 274778.52 Optimal expeditions: [(17, 1), (31, 2)]


### Results 

In order to balance the results for the different priors that I explored, my final choice was two expeditions on the tiles with multipliers $100$ and $82$. 

<img src="https://i.imgur.com/n7q6t4x.png" width="1200" />


Some of the true (i.e., ex post) shares are collected [here](https://docs.google.com/spreadsheets/d/1PlQlcJmFzcFJ_DV62cvzkVgphLiW0A5hLf7IwLlvYJQ/edit#gid=0).

In [ ]:
temp = pd.read_csv('data/shares.csv', sep=';', header=None).to_numpy()
dic = {mult:share for (mult, share) in temp}
shares_true = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        shares_true[i, j] = dic[arr[i, j][0]]
shares_true = shares_true/100
with np.printoptions(precision=3, suppress=True):
    print("True shares:", shares_true, sep='\n')

True shares:
[[0.015 0.082 0.019   nan 0.037]
 [0.03  0.062 0.098 0.041 0.012]
 [0.113 0.108 0.049 0.034 0.006]
 [0.046 0.054 0.065 0.054 0.026]
 [  nan   nan 0.019   nan   nan]]


Filling in the NaNs with equal values, the top 10 choices are displayed below.

In [ ]:
val_nan = (1-np.nansum(shares_true))/np.sum(np.isnan(shares_true))
maximize_prior_top(np.nan_to_num(shares_true, nan=val_nan),10)

[(106290.59586034171, [(80, 5), (52, 4)]),
 (105375.42951194028, [(90, 7), (52, 4)]),
 (105301.14425749943, [(80, 5), (90, 7)]),
 (103938.17304238147, [(52, 4), (30, 3)]),
 (103892.67636555269, [(35, 3), (52, 4)]),
 (103863.8877879406, [(80, 5), (30, 3)]),
 (103818.39111111182, [(80, 5), (35, 3)]),
 (103583.32345590748, [(41, 3), (52, 4)]),
 (103509.03820146661, [(41, 3), (80, 5)]),
 (102948.72143953915, [(90, 7), (30, 3)])]

The optimal decision was to go for two expeditions at the tiles with multipliers $80$ and $52$.